In [60]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [61]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle

In [62]:
with open('intents.json') as file:
    data = json.load(file)

In [63]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'],
   'context_set': ''},
  {'tag': 'age',
   'patterns': ['how old',
    'how old is Dheeraj',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': ['I am 22 years old!', '22 years young!'],
   'context_set': ''},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Dheeraj.',
    "I'm Dheeraj!",
    "I'm Dheeraj the AI student."],
   'context_set': ''},
  {'tag': 'shop',
   'patterns': ['Id like to buy something',
    'whats o

In [64]:
try:
    with open('data.pickle','rb') as f:
        data = json.load(file)
except:
    words = []
    labels = []
    docs_x = []
    docs_y =[]

    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [65]:
words

['Hi',
 'How',
 'are',
 'you',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Whats',
 'up',
 'cya',
 'See',
 'you',
 'later',
 'Goodbye',
 'I',
 'am',
 'Leaving',
 'Have',
 'a',
 'Good',
 'day',
 'how',
 'old',
 'how',
 'old',
 'is',
 'Dheeraj',
 'what',
 'is',
 'your',
 'age',
 'how',
 'old',
 'are',
 'you',
 'age',
 '?',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'I',
 'call',
 'you',
 'whats',
 'your',
 'name',
 '?',
 'Id',
 'like',
 'to',
 'buy',
 'something',
 'whats',
 'on',
 'the',
 'menu',
 'what',
 'do',
 'you',
 'reccommend',
 '?',
 'could',
 'i',
 'get',
 'something',
 'to',
 'eat',
 'when',
 'are',
 'you',
 'guys',
 'open',
 'what',
 'are',
 'your',
 'hours',
 'hours',
 'of',
 'operation']

In [66]:
labels

['greeting', 'goodbye', 'age', 'name', 'shop', 'hours']

In [67]:
docs_x

[['Hi'],
 ['How', 'are', 'you'],
 ['Is', 'anyone', 'there', '?'],
 ['Hello'],
 ['Good', 'day'],
 ['Whats', 'up'],
 ['cya'],
 ['See', 'you', 'later'],
 ['Goodbye'],
 ['I', 'am', 'Leaving'],
 ['Have', 'a', 'Good', 'day'],
 ['how', 'old'],
 ['how', 'old', 'is', 'Dheeraj'],
 ['what', 'is', 'your', 'age'],
 ['how', 'old', 'are', 'you'],
 ['age', '?'],
 ['what', 'is', 'your', 'name'],
 ['what', 'should', 'I', 'call', 'you'],
 ['whats', 'your', 'name', '?'],
 ['Id', 'like', 'to', 'buy', 'something'],
 ['whats', 'on', 'the', 'menu'],
 ['what', 'do', 'you', 'reccommend', '?'],
 ['could', 'i', 'get', 'something', 'to', 'eat'],
 ['when', 'are', 'you', 'guys', 'open'],
 ['what', 'are', 'your', 'hours'],
 ['hours', 'of', 'operation']]

In [68]:
docs_y

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'age',
 'age',
 'age',
 'age',
 'age',
 'name',
 'name',
 'name',
 'shop',
 'shop',
 'shop',
 'shop',
 'hours',
 'hours',
 'hours']

In [69]:
# word Stemming

In [70]:
words = [stemmer.stem(w.lower()) for w in words if w != '?']
words = sorted(list(set(words)))

labels = sorted(labels)

In [71]:
print(words)
print(len(words))

['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'dheeras', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'someth', 'the', 'ther', 'to', 'up', 'what', 'when', 'yo', 'you']
46


In [72]:
print(labels)

['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']


In [73]:
# Bag Of Words

In [74]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

In [75]:
len(bag)

46

In [76]:
output

[[0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0]]

In [77]:
bag

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [78]:
x

25

In [79]:
# convert training data and output to numpy arrays

In [80]:
training = numpy.array(training)
output = numpy.array(output)

In [81]:
len(training[2])

46

In [82]:
len(output)

26

In [83]:
# Developing a Model

In [84]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape = [None,len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [85]:
 # Training and Saving the Model

In [86]:
try:
    model.load('chatbot.tflearn')
except:
    model.fit(training,output,n_epoch = 1000, batch_size = 8, show_metric = True)
    model.save('chatbot.tflearn')

---------------------------------
Run id: G2J26N
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 26
Validation samples: 0
--


RuntimeError: Attempted to use a closed Session.

In [ ]:
model

In [ ]:
# Making Predictions

def bag_of_words(s,words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)

def chat():
    print('Start talking with the bot (type quit to stop)!')
    while True:
        inp = input('You : ')
        if inp.lower() == 'quit':
            break
        results = model.predict([bag_of_words(inp,words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(random.choice(responses))
        


In [ ]:
chat()